# <center>Hyperparameter Tuning for Conditional GAN (NLS-KDD)<center/>

In [1]:
import os
# 禁用 TensorFlow 的进度条等信息
import tensorflow as tf

# 禁用 TensorFlow 的标准日志
tf.get_logger().setLevel('ERROR')
# tf.compat.v1.disable_eager_execution() 


import sys
sys.path.append('../utils')
sys.path.append('../models')



import numpy as np
import pandas as pd
from collections import defaultdict
import pickle, itertools
from tqdm import tqdm  # tqdm 用于显示进度条
import matplotlib.pyplot as plt

import utils , preprocessing  



from keras.layers import Dense, Input, Dropout,concatenate
from keras.models import Model
from keras import backend as K
from scipy.stats import norm

### Check for GPU Availability

In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 11280145677751435289
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 6917455872
locality {
  bus_id: 1
  links {
  }
}
incarnation: 13706566146684718184
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1080, pci bus id: 0000:01:00.0, compute capability: 6.1"
xla_global_id: 416903419
]
Num GPUs Available:  1


## compute gradient penalty

In [3]:
def compute_gradient_penalty(D, real_samples, real_samples_label, fake_samples, fake_samples_label):
    # 去掉不必要的维度，将标签转换为 [batch_size, 1]
    real_samples_label = tf.squeeze(tf.expand_dims(real_samples_label, axis=1), axis=-1)
    fake_samples_label = tf.squeeze(tf.expand_dims(fake_samples_label, axis=1), axis=-1)

    # 合并特征和标签
    real_samples = tf.concat([real_samples, real_samples_label], axis=1)
    fake_samples = tf.concat([fake_samples, fake_samples_label], axis=1)

    # 获取随机权重
    alpha = tf.random.uniform([real_samples.shape[0], 1], minval=0, maxval=1, dtype=real_samples.dtype)
    alpha = tf.broadcast_to(alpha, tf.shape(real_samples))

    # 确保 real_samples 和 fake_samples 是 float32 类型
    real_samples = tf.cast(real_samples, tf.float32)
    fake_samples = tf.cast(fake_samples, tf.float32)
    alpha = tf.cast(alpha, tf.float32)

    # 对真实样本和生成样本进行插值
    interpolates = alpha * real_samples + (1 - alpha) * fake_samples
    interpolates = tf.Variable(interpolates, trainable=True)  # 转换为可求导的变量

    # 使用 GradientTape 计算梯度
    with tf.GradientTape() as tape:
        tape.watch(interpolates)
        
        # 分离特征和标签
        features = interpolates[:, :-1]
        label = tf.cast(interpolates[:, -1], tf.float32)
        
        # 拼接 features 和 label
        combined_input = tf.concat([features, tf.expand_dims(label, axis=1)], axis=1)
        d_interpolates = D(combined_input)

    # 创建中间变量
    fake = tf.ones_like(d_interpolates)

    # 计算插值样本输出的梯度
    gradients = tape.gradient(d_interpolates, interpolates)
    if gradients is None:
        raise ValueError("gradients is None, check `tape.watch(interpolates)` ")

    # 计算梯度的范数
    gradients = tf.reshape(gradients, [tf.shape(gradients)[0], -1])
    gradient_penalty = tf.reduce_mean((tf.norm(gradients, ord=2, axis=1) - 1) ** 2)
    
    return gradient_penalty


In [4]:
def plot_summary(d_l, g_l,acc_r,acc_g, m =''):
    # n = np.arange(len(d_l))
    # 确保 d_l 和 g_l 的长度一致
    # min_length = min(len(d_l), len(g_l), len(acc_r), len(acc_g))
    # d_l = d_l[:min_length]
    # g_l = g_l[:min_length]
    # acc_r = acc_r[:min_length]
    # acc_g = acc_g[:min_length]

    # 找出最大长度
    max_length = max(len(d_l), len(g_l), len(acc_r), len(acc_g))

    # 用最后一个值填充较短序列
    d_l = np.pad(d_l, (0, max_length - len(d_l)), 'edge')
    g_l = np.pad(g_l, (0, max_length - len(g_l)), 'edge')
    acc_r = np.pad(acc_r, (0, max_length - len(acc_r)), 'edge')
    acc_g = np.pad(acc_g, (0, max_length - len(acc_g)), 'edge')

    n = np.arange(max_length)  # 保证 n 与数据长度一致

    title = 'Loss and Accuracy plot'+'\n'+ m
    title = title.replace('.pickle','')
    fig, axs = plt.subplots(2,figsize=(19.20,10.80))

    axs[0].set_title(title,fontsize=20.0,fontweight="bold")
    axs[0].plot(n, g_l,label='Generator loss',linewidth=4)
    axs[0].plot(n, d_l,label='Discriminator loss',linewidth=4)
    axs[0].legend(loc=0, prop={'size': 20})
    axs[0].set_ylabel('Loss',fontsize=20.0,fontweight="bold")
    axs[0].tick_params(labelsize=20)
    axs[0].tick_params(axis='x',which='both',bottom=False,top=False,labelbottom=False,labelsize=20)

    # axs[1].plot(n, acc,'r',label='Discriminator accuracy',linewidth=4)
    axs[1].plot(n, acc_g,label='Accuracy on Generated',linewidth=4)
    axs[1].plot(n, acc_r,label='Accuracy on Real',linewidth=4)
    axs[1].legend(loc=0,prop={'size': 20})
    axs[1].set_ylabel('Accuracy',fontsize=20.0,fontweight="bold")
    axs[1].set_xlabel('Ephoc',fontsize=20.0,fontweight="bold")
    axs[1].tick_params(labelsize=20)

    plt.tight_layout()
    #plt.show()
    if not os.path.exists("imgs"):
        os.makedirs('imgs')
    plt.savefig(f'imgs/{m[:-7]}.png',dpi = 300)
    plt.close('all') #plt.close(fig)

## Conditional Generative Adversarial Network Class definition

In [5]:
class WCGAN():
    """Conditinal Generative Adversarial Network class"""
    
    def __init__(self,arguments,X,y):
        
        if not tf.config.list_physical_devices('GPU'):
            self.device = tf.device('/CPU:0')
            
        else:
            self.device = tf.device('/GPU:0')
            

        [self.rand_noise_dim, self.tot_epochs, self.batch_size,self.D_epochs, \
         self.G_epochs,self.G_learning_rate, self.D_learning_rate, self.n_layers, self.activation,self.optimizer, self.min_num_neurones,self.lambda_gp] = arguments
      
        self.X_train = X
        self.y_train = y
        
        self.label_dim = y.shape[1]
        self.x_data_dim = X.shape[1]

        self.g_losses = []
        self.d_losses, self.disc_loss_real, self.disc_loss_generated, self.gp_losses = [], [], [], []
        self.acc_history = []
        
        self.__define_models()
        self.gan_name = '_'.join(str(e) for e in arguments).replace(".","")
        
        self.terminated = False

    def build_generator(self,x,labels):
        """Create the generator model G(z,l) : z -> random noise , l -> label (condition)"""
        
        x = concatenate([x,labels])
        for i in range(1,self.n_layers+1):
            x = Dense(self.min_num_neurones*i, activation=self.activation)(x)
            
        x = Dense(self.x_data_dim)(x)
        x = concatenate([x,labels])

        return x

    def build_discriminator(self,x):
        """Create the discrimnator model D(G(z,l)) : z -> random noise , l -> label (condition)"""
        
        for n in reversed(range(1,self.n_layers+1)):
            x = Dense(self.min_num_neurones*n, activation=self.activation)(x)
        
        x = Dense(1, activation='sigmoid')(x)

        return x

    def __define_models(self):
        """Define Generator, Discriminator & combined model"""
        
        # Create & Compile generator
        generator_input = Input(shape=(self.rand_noise_dim,))
        labels_tensor = Input(shape=(self.label_dim,))
        generator_output = self.build_generator(generator_input, labels_tensor)

        self.generator = Model(inputs=[generator_input, labels_tensor], outputs=[generator_output], name='generator')
        self.generator.compile(loss='binary_crossentropy',optimizer=self.optimizer, metrics=['accuracy'])
        K.set_value(self.generator.optimizer.lr,self.G_learning_rate)
        

        # Create & Compile generator
        discriminator_model_input = Input(shape=(self.x_data_dim + self.label_dim,))
        discriminator_output = self.build_discriminator(discriminator_model_input)

        self.discriminator = Model(inputs=[discriminator_model_input],outputs=[discriminator_output],name='discriminator')
        self.discriminator.compile(loss='binary_crossentropy',optimizer=self.optimizer, metrics=['accuracy'])
        K.set_value(self.discriminator.optimizer.lr,self.D_learning_rate)

        # Build "frozen discriminator"
        frozen_discriminator = Model(inputs=[discriminator_model_input],outputs=[discriminator_output],name='frozen_discriminator')
        frozen_discriminator.trainable = False

        # Debug 1/3: discriminator weights
        n_disc_trainable = len(self.discriminator.trainable_weights)

        # Debug 2/3: generator weights
        n_gen_trainable = len(self.generator.trainable_weights)

        # Build & compile combined model from frozen weights discriminator
        combined_output = frozen_discriminator(generator_output)
        self.combined = Model(inputs = [generator_input, labels_tensor],outputs = [combined_output],name='adversarial_model')
        self.combined.compile(loss='binary_crossentropy',optimizer=self.optimizer, metrics=['accuracy'])
        K.set_value(self.combined.optimizer.lr,self.D_learning_rate)

        # Debug 3/3: compare if trainable weights correct
        assert(len(self.discriminator.trainable_weights) == n_disc_trainable)
        assert(len(self.combined.trainable_weights) == n_gen_trainable)
        
    def __get_batch_idx(self):
        """random selects batch_size samples indexes from training data"""
        
        batch_ix = np.random.choice(len(self.X_train), size=self.batch_size, replace=False)

        return batch_ix
    
    def save_model(self,model_dir = "./models"):
        if not os.path.exists(model_dir):
            os.makedirs(model_dir)

         # 保存生成器
        self.generator.save(f"{model_dir}/{self.gan_name}_generator_model.h5")

        # 保存判别器
        self.discriminator.save(f"{model_dir}/{self.gan_name}_discriminator_model.h5")

        # 保存对抗模型（combined model）
        self.combined.save(f"{model_dir}/{self.gan_name}_combined_model.h5")
    
    def plot_summary(self, d_l, g_l,acc_r,acc_g, m):

        # 找出最大长度
        max_length = max(len(d_l), len(g_l), len(acc_r), len(acc_g))

        # 用最后一个值填充较短序列
        d_l = np.pad(d_l, (0, max_length - len(d_l)), 'edge')
        g_l = np.pad(g_l, (0, max_length - len(g_l)), 'edge')
        acc_r = np.pad(acc_r, (0, max_length - len(acc_r)), 'edge')
        acc_g = np.pad(acc_g, (0, max_length - len(acc_g)), 'edge')

        n = np.arange(max_length)  # 保证 n 与数据长度一致

        title = 'Loss and Accuracy plot'+'\n'+ m
        title = title.replace('.pickle','')
        fig, axs = plt.subplots(2,figsize=(19.20,10.80))

        axs[0].set_title(title,fontsize=20.0,fontweight="bold")
        axs[0].plot(n, g_l,label='Generator loss',linewidth=4)
        axs[0].plot(n, d_l,label='Discriminator loss',linewidth=4)
        axs[0].legend(loc=0, prop={'size': 20})
        axs[0].set_ylabel('Loss',fontsize=20.0,fontweight="bold")
        axs[0].tick_params(labelsize=20)
        axs[0].tick_params(axis='x',which='both',bottom=False,top=False,labelbottom=False,labelsize=20)

        # axs[1].plot(n, acc,'r',label='Discriminator accuracy',linewidth=4)
        axs[1].plot(n, acc_g,label='Accuracy on Generated',linewidth=4)
        axs[1].plot(n, acc_r,label='Accuracy on Real',linewidth=4)
        axs[1].legend(loc=0,prop={'size': 20})
        axs[1].set_ylabel('Accuracy',fontsize=20.0,fontweight="bold")
        axs[1].set_xlabel('Ephoc',fontsize=20.0,fontweight="bold")
        axs[1].tick_params(labelsize=20)

        plt.tight_layout()
        #plt.show()
        if not os.path.exists("imgs"):
            os.makedirs('imgs')
        plt.savefig(f'imgs/{m[:-7]}.png',dpi = 300)
        plt.close('all') #plt.close(fig)
    
    def dump_to_file(self,save_dir = "./logs"):
        """Dumps the training history and GAN config to pickle file """
        
        H = defaultdict(dict)
        H["acc_history"] = self.acc_history
        H["Generator_loss"] = self.g_losses
        H["disc_loss_real"] = self.disc_loss_real
        H["disc_loss_gen"] = self.disc_loss_generated
        H["discriminator_loss"] = self.d_losses
        H["rand_noise_dim"] , H["total_epochs"] = self.rand_noise_dim, self.tot_epochs
        H["batch_size"] , H["g_learning_rate"], H["d_learning_rate"]  = self.batch_size, self.G_learning_rate, self.D_learning_rate
        H["n_layers"] , H["activation"]  = self.n_layers, self.activation
        H["optimizer"] , H["min_num_neurones"] = self.optimizer, self.min_num_neurones

        # save images
        with open(f"{save_dir}/WCGAN_{self.gan_name}.pickle", "wb") as output_file:
            pickle.dump(H, output_file)

        with open(f"{save_dir}/WCGAN_{self.gan_name}.pickle", 'rb') as f:
            x = pickle.load(f)

        d_l = np.array(x['discriminator_loss']).ravel()
        g_l = np.array(x['Generator_loss']).ravel()
        acc_history = np.array(x['acc_history'])
        acc = acc_history.sum(axis=1) * 0.5
        acc_real = acc_history[:,1]
        acc_gen = acc_history[:,0]

        filename = f"WCGAN_{self.gan_name}.pickle"
        self.plot_summary(d_l, g_l,acc_real,acc_gen,filename)
        
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
        
        # with open(f"{save_dir}/{self.gan_name}{'.pickle'}", "wb") as output_file:
        #     pickle.dump(H,output_file)
        
        self.save_model()

    def train(self):
        """Trains the WCGAN model"""

        # Adversarial ground truths
        real_labels = np.ones((self.batch_size, 1))
        fake_labels = np.zeros((self.batch_size, 1))
        # Adversarial ground truths with noise
        #real_labels = np.random.uniform(low=0.999, high=1.0, size=(self.batch_size,1))
        #fake_labes = np.random.uniform(low=0, high=0.00001, size=(self.batch_size,1))
        
        for epoch in range(self.tot_epochs):
            #Train Discriminator
            for i in range(self.D_epochs):

                idx = self.__get_batch_idx()
                x, labels = self.X_train[idx], self.y_train[idx]

                #Sample noise as generator input
                noise = np.random.normal(0, 1, (self.batch_size, self.rand_noise_dim))

                #Generate a half batch of new images
                generated_x = self.generator.predict([noise, labels], verbose=0)
                
                #Train the discriminator
                d_loss_fake = self.discriminator.train_on_batch(generated_x, fake_labels)
                d_loss_real = self.discriminator.train_on_batch(np.concatenate((x,labels),axis=1), real_labels)
                
                # Compute gradient penalty
                gp = compute_gradient_penalty(self.discriminator, x, labels, generated_x[:, :-1], fake_labels)
                self.gp_losses.append(gp)

                d_loss = 0.5 * np.add(d_loss_real,d_loss_fake) + self.lambda_gp * gp

            self.disc_loss_real.append(d_loss_real[0])
            self.disc_loss_generated.append(d_loss_fake[0])
            self.d_losses.append(d_loss[0])
            self.acc_history.append([d_loss_fake[1],d_loss_real[1]])

            #NB: Gradients could be exploding or vanishing (cliping or changing activation function,learning rate could be a solution)
            # if np.isnan(d_loss_real) or np.isnan(d_loss_fake):
            #     self.terminated = True
            #     break
            #NB: Gradients could be exploding or vanishing (cliping or changing activation function, learning rate could be a solution)
            if np.isnan(d_loss_real).any() or np.isnan(d_loss_fake).any():
                self.terminated = True
                break

                
            #Train Generator (generator in combined model is trainable while discrimnator is frozen)
            for j in range(self.G_epochs):
                #Condition on labels
                # sampled_labels = np.random.randint(1, 5, self.batch_size).reshape(-1, 1)
                sampled_labels = np.random.choice([0,2,3,4],(self.batch_size,1), replace=True)

                #Train the generator
                g_loss = self.combined.train_on_batch([noise, sampled_labels], real_labels)
                self.g_losses.append(g_loss[0])

            #Print metrices


            
            # print ("Epoch : {:d} [D loss: {:.4f}, acc.: {:.4f}] [G loss: {:.4f}]".format(epoch, d_loss[0], 100*d_loss[1], g_loss[0]))


## Read & Preprocess Input data

In [6]:
train,test, label_mapping = preprocessing.get_data(encoding="Label")
data_cols = list(train.columns[ train.columns != 'label' ])
x_train , x_test = preprocessing.preprocess(train,test,data_cols,"Robust",True)

y_train = x_train.label.values
y_test = x_test.label.values

data_cols = list(x_train.columns[ x_train.columns != 'label' ])

to_drop = preprocessing.get_contant_featues(x_train,data_cols)
x_train.drop(to_drop, axis=1,inplace=True)
x_test.drop(to_drop, axis=1,inplace=True)

data_cols = list(x_train.columns[ x_train.columns != 'label' ])

att_ind = np.where(x_train.label != label_mapping["normal"])[0]
x = x_train[data_cols].values[att_ind]
y = y_train[att_ind].reshape(-1,1)

d:\WorkSpace\GAN\GANs_for_Network_Intrusion_Data\NSL-KDD\notebooks
True


## Set parameters to tune

In [7]:
# rand_dim = np.arange(10,110,10)
# base_n_count = np.arange(3,41,3)
# ephocs = np.arange(100,5000,100)
# batch_sizes = [64,128,250,300,350]
# learning_rates = np.logspace(-1,-4,num=20)
# num_layers = np.arange(3,20)

# optimizers = ["sgd", "RMSprop", "adam", "Adagrad", "Adamax","Nadam"]
# activation_func = ["tanh","relu","softplus","linear","elu"]

#create a logs directory
if not os.path.exists('logs'):
    os.makedirs('logs')


# rand_dim = np.arange(10,110,10)
# base_n_count = np.arange(5,51,5)  # 调整基本神经元数量的范围，从3-41变为5-51，步长为5
# ephocs = np.arange(500,5500,500)  # 调整周期范围，从100-5000变为500-5500，步长为500
# batch_sizes = [128, 256, 512]  # 简化批量大小选择为128, 256, 512
# learning_rates = np.logspace(-2,-5,num=10)  # 调整学习率范围，从0.01到0.00001，共10个值
# num_layers = np.arange(2,10)  # 调整隐藏层数量，范围从2到10

# optimizers = ["adam", "RMSprop", "Nadam"]  # 精简优化器选项，去除一些效果可能较差的
# activation_func = ["relu", "tanh", "elu"]  # 精简激活函数选项

# rand_dim = np.arange(10,110,10)
base_n_count = [27]  # 调整基本神经元数量的范围，从3-41变为5-51，步长为5
ephocs = [1000]  # 调整周期范围，从100-5000变为500-5500，步长为500
batch_sizes = [256]  # 简化批量大小选择为128, 256, 512
# learning_rates = [[0.00001], [0.00005]]  # G_learning_rates, D_learning_rates 调整学习率范围，从0.01到0.00001，共10个值
G_learning_rates = np.arange(0.00002,0.00005,0.00001)
G_learning_rates = np.append(G_learning_rates, [0.000005, 0.0005])
D_learning_rates = [0.00005]
num_layers = [4]  # 调整隐藏层数量，范围从2到10
lambda_gp = [2]  # 调整lambda_gp的值1
optimizers = ["adam"]  # 精简优化器选项，去除一些效果可能较差的
activation_func = ["tanh"]  # 精简激活函数选项
d_ephocs = [2]
g_ephocs = [1]


### Using itertools

In [8]:
tot = list(itertools.product([32], ephocs, batch_sizes, d_ephocs, g_ephocs,
                             G_learning_rates, D_learning_rates, num_layers, activation_func, optimizers, base_n_count,lambda_gp))

# 遍历所有组合并训练模型
for idx, i in tqdm(enumerate(tot), total=len(tot), desc="Processing"):
    args = list(i)
    wcgan = WCGAN(args, x, y)  # 假设 CGAN 类已经定义好
    wcgan.train()  # 开始训练
    if not wcgan.terminated:  # 检查训练是否终止
        wcgan.dump_to_file()  # 保存训练结果

Processing:   0%|          | 0/5 [01:44<?, ?it/s]


KeyboardInterrupt: 

In [ ]:


# WCGAN_32_1000_256_2_1_5e-05_4_tanh_adam_27_2

# WCGAN_32_1000_256_2_1_5e-06_5e-05_4_tanh_adam_27_2